In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
import pandas as pd
import xarray as xr
import rioxarray

import codebase

## Checking that load functions work
The full load and subset load both work.

In [2]:
dem_test , lat_test , lon_test = codebase.load_data.load_DEM_full_as_nparray()
dem_DA = codebase.load_data.load_DEM_full_as_rxrDA()

In [3]:
bbox_vals = pd.DataFrame(data = np.array([10,15,30,25]).reshape(1,-1),columns = ['minx', 'miny', 'maxx', 'maxy'])
bbox_vals

,minx,miny,maxx,maxy
0,10,15,30,25


In [4]:
sub_test , lat_test , lon_test= codebase.load_data.load_DEM_subset_as_nparray(bbox_vals)
sub_DA = codebase.load_data.load_DEM_subset_as_rxrDA(bbox_vals)

NameError: name 'load_DEM_full' is not defined

## From np.array to xr.DataArray

In [ ]:
pd.Index(lat_test.round(5))

In [ ]:
lat_grid ,lon_grid = np.meshgrid(lat_test,lon_test,indexing='ij')


In [ ]:
full_xr = xr.DataArray(data = dem_test , dims = ['lat','lon'], 
                       coords=dict(lat= (['lat'],lat_test),lon=(['lon'],lon_test)))
# test_xr = xr.DataArray(data = sub_test , dims = ['lat','lon'],
#                        coords=dict(lat= (['lat','lon'],lat_grid),lon=(['lat','lon'],lon_grid)))
full_xr

In [ ]:
global_rxr = dem_DA.rio.write_crs(4326)
global_rxr.rio.set_spatial_dims("lon", "lat", inplace=True)

In [ ]:
clipped_rxr = global_rxr.rio.clip_box(*bbox_vals.values[0])

## Loading DEM data (not as a function)

In [ ]:
dem_filepath = '/global/scratch/users/cgerlein/fc_ecohydrology_scratch/CYGNSS/Data/'
dem_filename = 'CYGNSS_0_01_deg_Map_DEM_Mask.npy'
dem = np.load(dem_filepath+dem_filename)

In [ ]:
# latitude ranges from -45 to 45, with num = 9001  
# longitude ranges from -180 to 180 with num = 36001
# thus the coordinates can be set up as follow: 
lat = np.linspace(-45,45,dem.shape[0])
lon = np.linspace(-180,180,dem.shape[1])
# and you can cut off the data by lat and lon  

In [ ]:
print(dem.shape)

## Messing with DEM data stats

In [ ]:
dem.max() , dem.min() , dem.mean()

In [ ]:
noam = dem[6000:,5000:10000]
tibet = dem[5000:8000,25000:30000]

In [ ]:
plt.imshow(tibet,origin='lower',cmap = 'terrain',vmax = 10000)
# plt.imshow(tibet,origin='lower',cmap = 'jet',vmin = 30000, vmax = 32000)
plt.colorbar()

In [ ]:
ocean_mask = dem >= 10000
land_mask = ~ocean_mask

In [ ]:
plt.figure(figsize=(60,15))
plt.rcParams.update({'font.size': 14})
# plt.imshow(ocean_mask,origin='lower')
plt.imshow(dem,origin = "lower", cmap="terrain", vmax=7000)
plt.colorbar(location = "bottom")

In [ ]:
lat_grid , lon_grid = np.meshgrid(lat,lon,indexing = 'ij',copy=False)

In [ ]:
dem.shape

In [ ]:
test_bool = ((lat_grid > 0) * (lon_grid >0))
print(test_bool.shape)

In [ ]:
lat[lat >= 0]

In [ ]:
dem[test_bool].shape

In [ ]:
## Testing out using lat/lon to subset
lat_subset = (lat_grid > 0)
lon_subset = (lon > 0)

subset_test = dem[np.ix_(lat>0,(lon > 0)&(lon<90))]
print(subset_test.shape)
plt.imshow(subset_test,origin = "lower", cmap="terrain", vmax=6000)